In [1]:
import pandas as pd
import urllib
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import sessionmaker

# Custom upload with connection string
from engine_info import server_info
from retailers_normalized_data import RetailerProducts, Retailers, Date, RetailerPrices, db #MasterDate

import warnings
warnings.filterwarnings('ignore')

C:\Users\UtopianNinja\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [2]:
import numpy as np

In [3]:
# Creating a connection to MS SQL SERVER
params = urllib.parse.quote_plus(server_info)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
connection = engine.connect()

In [4]:
# Check what is in the database
engine.table_names()

['Capetown_Fresh_produce_market',
 'Capetown_Fresh_produce_market_cleaned',
 'container',
 'Durban_Fresh_produce_market',
 'Durban_Fresh_produce_market_cleaned',
 'inventory',
 'Joburg_Fresh_produce_combined_cleaned',
 'Joburg_Fresh_produce_commodity_cleaned',
 'Joburg_Fresh_produce_commodity_raw',
 'Joburg_Fresh_produce_container_cleaned',
 'Joburg_Fresh_produce_container_raw',
 'Joburg_Fresh_produce_product_combination_raw',
 'Joburg_Fresh_produce_scrapping_date',
 'master_date',
 'PickNPay_Prices',
 'PickNPay_Prices_cleaned',
 'product',
 'product_combination',
 'sales',
 'Shoprite_Prices',
 'Shoprite_Prices_cleaned',
 'sysdiagrams',
 'woolworths_Prices',
 'Woolworths_Prices_cleaned']

In [5]:
metadata = MetaData(bind=engine)

In [6]:
db.create_all()

## 1. Retailers

In [7]:
retailers_df = pd.read_sql_table('Retailers', con=engine)

In [8]:
retailers_df.head()

,id,RetailerName


In [9]:
# Check the data type so that it's suited to be inserted in a normalized database
retailers_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            0 non-null      int64 
 1   RetailerName  0 non-null      object
dtypes: int64(1), object(1)
memory usage: 0.0+ bytes


In [10]:
Shoprite = Retailers(RetailerName="Shoprite")
PnP = Retailers(RetailerName="Pick 'n Pay")
Woolworths = Retailers(RetailerName="Woolworths")
db.session.add(Shoprite)
db.session.add(PnP)
db.session.add(Woolworths)
db.session.commit()

In [11]:
retailers_df = pd.read_sql_table('Retailers', con=engine)

In [12]:
retailers_df.head()

,id,RetailerName
0,2,Pick 'n Pay
1,1,Shoprite
2,3,Woolworths


## 2. Date

In [13]:
date_df = pd.read_sql_table('Date', con=engine)

In [14]:
date_df.head()

,id,date_key


In [16]:
pnp_df = date_df = pd.read_sql_table('PickNPay_Prices_cleaned', con=engine)
shoprite_df = pd.read_sql_table('Shoprite_Prices_cleaned', con=engine)
woolworths_df = pd.read_sql_table('Woolworths_Prices_cleaned', con=engine)

In [18]:
pnp_df.head()

,index,item,Price,Date
0,0,PnP Oranges 4s,24.99,2020-08-24
1,1,PnP Plums 750g,34.99,2020-08-24
2,2,PnP Starking Apples 1.5kg,29.99,2020-08-24
3,3,PnP Baby Papaya 2s,29.99,2020-08-24
4,4,PnP Kids Yellow Apples 1kg,17.99,2020-08-24


In [19]:
shoprite_df.head()

,index,item,Price,Date
0,0,Grapefruit In Bag 2kg,24.99,2020-08-25
1,1,Safari Pitted Dates Pack 250g,16.99,2020-08-25
2,2,Lemon Per kg,18.99,2020-08-25
3,3,Naartjie Per kg,19.99,2020-08-25
4,4,Grapefruit Per kg,14.99,2020-08-25


In [20]:
woolworths_df.head()

,index,item,Price,Date
0,0,Queen Pineapples 2Pk,32.99,2020-08-31
1,1,Four Leaf Salad 180g,24.99,2020-08-31
2,2,Raspberries 160g,39.99,2020-08-31
3,3,Portabellini Mushrooms 250g,27.99,2020-08-31
4,4,Colour Peppers 3Pk,36.99,2020-08-31


In [24]:
all_dates = pd.DataFrame()
all_dates["Date"] = pnp_df["Date"]
all_dates["Date"] = all_dates["Date"].append(shoprite_df["Date"])
all_dates["Date"] = all_dates["Date"].append(woolworths_df["Date"])
all_dates = all_dates.unique()

ValueError: cannot reindex from a duplicate axis

In [ ]:
for date in all_dates:
    datum = Date(date_key=date)
    db.session.add(datum)
    db.session.commit()

In [ ]:
date_df = pd.read_sql_table('Date', con=engine)

In [ ]:
date_df.head()

## 3. RetailerProducts (JHB)

In [ ]:
products_df = pd.read_sql_table('RetailerProducts', con=engine)

In [17]:
products_df.head()

NameError: name 'products_df' is not defined

In [ ]:
all_products = pnp_df[pnp_df["item"]]["item"] + shoprite_df['item'] + woolworths_df['item']
all_products = all_products.unique()

In [ ]:
for item in all_products:
    product = RetailerProducts(RetailerProduct=item)
    db.session.add(product)
    db.session.commit()

In [ ]:
products_df = pd.read_sql_table('RetailerProducts', con=engine)

In [ ]:
products_df.head()

In [ ]:
stop

In [ ]:
db.session.close()

In [ ]:
connection.close()